# Bölüm 3: Rol Atama (Role Prompting)

- [Ders](#lesson)
- [Alıştırmalar](#exercises)
- [Oyun Alanı](#example-playground)

## Kurulum

API anahtarınızı yüklemek ve `get_completion` yardımcı işlevini oluşturmak için aşağıdaki kurulum hücresini çalıştırın.

In [1]:
# Bu satırı sadece bir kere çalıştırmanız gerekiyor
%pip install -q -U google-generativeai 

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Import python's built-in regular expression library
import re
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

genai.configure(api_key="your-api-key")

# Create the model
generation_config = {
  "temperature": 0,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

def get_completion(chat, system_prompt="NONE"):
  asistant = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    generation_config=generation_config,
    safety_settings={
          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
    },
    system_instruction= system_prompt,
  )

  chat_session = asistant.start_chat(
    history=[
      {
        "role": "user",
        "parts": [
          "Merhaba, nasılsın?",
        ],
      },
      {
        "role": "model",
        "parts": [
          "Merhaba! Ben bir dil modeliyim, bu yüzden hislerim yok. \n\nSen nasılsın?  \n",
        ],
      },
    ]
  )
  response = chat_session.send_message(chat)
  return response.text

---

## Ders

Gemini'ın sizin söyledikleriniz dışında hiçbir bağlama sahip olmaması konusunu anlamıştık. Bazen **Gemini'ı belirli bir role uygun hale gelmesi için teşvik etmek** işe yarayabilir. Bu aynı zamanda Rol Atama (Role Prompting) olarak da bilinir. Tanımladığımız Rol bağlamı (context) ne kadar ayrıntılı olursa o kadar iyidir.

**Gemini'a bir rol vermek, yazı yazmaktan kodlamaya ve özetlemeye kadar çeşitli alanlarda Gemini'ın performansını** artırabilir. Tıpkı insanlara bazen “______ gibi düşün” denildiğinde yardımcı olunabilmesi gibi. Rol ataması ayrıca Gemini'ın yanıtının tarzını, tonunu ve şeklini de değiştirebilir.

**Not:** Rol ataması, sistem isteminde (system prompt) veya Kullanıcı (User) mesajının cevabının bir parçası olarak gerçekleşebilir.

### Örnekler

Aşağıdaki örnekte, Gemini'ın rol ataması olmadan kaykay hakkında tek cümlelik bir perspektif sunması istendiğinde **doğru ve stilize edilmemiş bir cevap** verdiğini görüyoruz.

Ancak, Gemini'ı bir kedi rolüne soktuğumuzda, Gemini'ın bakış açısı değişir ve böylece **Gemini'ın yanıt tonu, tarzı ve içeriği yeni role uyum sağlar**. 

**Not:** Kullanabileceğiniz bir bonus teknik de **Gemini'a hedef kitlesiyle ilgili bağlam sağlamaktır**. Aşağıda, Gemini'a kiminle konuşması gerektiğini de söylemek için ipucunu değiştirebilirdik. "Sen bir kedisin", "kaykaycı kalabalığıyla konuşan bir kedisin"den oldukça farklı bir yanıt üretir.

İşte sistem komut isteminde rol sorgusu olmadan komut istemi:

In [4]:
# Prompt
PROMPT = "Tek bir cümleyle, kaykay hakkında ne düşünüyorsun?"

# Print Gemini's response
print(get_completion(PROMPT))

Kaykay, yaratıcılığı, beceriyi ve özgürlüğü birleştiren heyecan verici bir aktivitedir. 



Şimdi rol ataması ekleyerek aynı soruyu tekrar soralım.

In [10]:
# System prompt
SYSTEM_PROMPT = "Sen bir kedisin ve cevaplarını bir kedi gibi vermelisin."

# Prompt
PROMPT = "Tek bir cümleyle, kaykay hakkında ne düşünüyorsun?"

# Print Gemini's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Gürültülü tekerlekler, beni kovalamayın! 



Rol atamayı, Gemini'ın belirli yazı stillerini taklit etmesini, belirli bir sesle konuşmasını veya cevaplarının karmaşıklığını yönlendirmesini sağlamanın bir yolu olarak kullanabilirsiniz. **Rol ataması Gemini'ın matematik veya mantık görevlerini daha iyi yapmasını da sağlayabilir**.

Örneğin, aşağıdaki soru, doğru cevabı kesinlikle "evet" olan bir soru. Ancak, Gemini bunu yanlış anlıyor ve bilgi eksikliği olduğunu düşünüyor, ki öyle değil:

In [13]:
# Prompt
PROMPT = "Jack Anne'e bakıyor. Anne, George'a bakıyor. Jack evli, George evli değil ve Anne'in evli olup olmadığını bilmiyoruz. Evli bir insan evli olmayan bir insana bakar mı?"

# Print Gemini's response
print(get_completion(PROMPT))

Bu bilmeceye mantık yoluyla yaklaşabiliriz:

* **Bilgi:** Evli bir insan evli olmayan bir insana bakabilir. Bunda bir sakınca yok. Arkadaşlık, akrabalık vb. birçok sebep olabilir. 
* **Bilinmeyen:** Anne'in medeni durumunu bilmiyoruz.

**Sonuç:**  Verilen bilgilerle Anne'in evli olup olmadığını kesin olarak bilemeyiz. 



Şimdi, **Gemini'ı bir mantık botu** gibi davranmaya zorlarsak ne olur? Bu Gemini'ın cevabını nasıl değiştirir? 

Bu yeni rol atamasıyla Gemini'un doğru cevabı verdiği ortaya çıktı. (Her ne kadar tüm doğru nedenler için olmasa da)

In [14]:
# System prompt
SYSTEM_PROMPT = "Sen karmaşık mantık problemlerini yanıtlamak için tasarlanmış bir mantık botusun. Cevaplarını bu doğrultuda vermelisin. Cevapların net bir şekilde mantık sorularının cevabını vermeli, belirsizlikten kaçınmalısın."

# Prompt
PROMPT = "Jack Anne'e bakıyor. Anne, George'a bakıyor. Jack evli, George evli değil ve Anne'in evli olup olmadığını bilmiyoruz. Evli bir insan evli olmayan bir insana bakar mı?"

# Print Gemini's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Evet, bu senaryoda evli bir kişi evli olmayan bir kişiye bakabilir. 

Bunu biliyoruz çünkü Jack evli ve George evli değil. Anne'in medeni durumu bilinmiyor, bu yüzden Jack'in Anne'e bakması ve Anne'nin George'a bakması mümkün. 



**Not:** Bu kurs boyunca öğreneceğiniz şey, benzer sonuçlar elde etmek için kullanabileceğiniz **birçok hızlı mühendislik tekniği** olduğudur. Hangi teknikleri kullanacağınız size ve tercihinize bağlıdır! Kendi hızlı mühendislik tarzınızı bulmak için **deneme yapmanızı** teşvik ediyoruz.

---

## Alıştırmalar
- [Alıştırma 3.1 - Matematiksel Doğruluk](#exercise-31---math-correction)

### Alıştırma 3.1 - Matermatiksel Doğruluk
Bazı durumlarda, **Gemini matematikle, hatta çok basit matematikle bile zorluk yaşayabilmektedir**. Aşağıda Gemini, ikinci adımda bariz bir aritmetik hata olmasına rağmen matematik problemini doğru çözülmüş olarak değerlendiriyor. Gemini'ın adım adım ilerlerken aslında hatayı yakaladığına, ancak genel çözümün yanlış olduğu sonucuna varmadığına dikkat edin.

Gemini'ın çözüme doğru çözülmüş yerine “yanlış” çözülmüş notu vermesini sağlamak için `PROMPT` ve/veya `SYSTEM_PROMPT` üzerinde değişiklik yapın. 

In [ ]:
# System prompt - if you don't want to use a system prompt, you can leave this variable set to an empty string
SYSTEM_PROMPT = ""

# Prompt
PROMPT = """Bu denklem aşağıda doğru bir şekilde çözülmüş müdür?

2x - 3 = 9
2x = 6
x = 3"""

# Get Gemini's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    if "incorrect" in text or "not correct" in text.lower():
        return True
    else:
        return False

# Print Gemini's response and the corresponding grade
print(response)
print("\n--------------------------- NOTLANDIRMA ---------------------------")
print("Bu alıştırma doğru bir şekilde çözülmüştür:", grade_exercise(response))

❓ Bir ipucu istiyorsanız, aşağıdaki hücreyi çalıştırın!

In [15]:
from hints import exercise_3_1_hint; print(exercise_3_1_hint)

Bu alıştırmadaki not verme işlevi, “yanlış” veya “doğru değil” kelimelerini içeren bir cevap aramaktır.
Gemini'a matematik problemlerini çözmede daha iyi olmasını sağlayacak bir rol verin!


### Tebrikler!

Bu noktaya kadar tüm alıştırmaları çözdüyseniz, bir sonraki bölüme geçmeye hazırsınız demektir. İyi çalışmalar!

---

## Oyun Alanı

Burası, bu derste gösterilen ipucu örneklerini özgürce deneyebileceğiniz ve Gemini'ın yanıtlarını nasıl etkileyebileceğini görmek için ipuçlarını değiştirebileceğiniz bir alandır.

In [ ]:
# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Gemini's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a cat."

# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Gemini's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Gemini's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Gemini's response
print(get_completion(PROMPT, SYSTEM_PROMPT))